In [0]:

import pandas as pd
import json
import sys
import time

In [56]:
!pip install surprise

     |████████████████████████████████| 3.3MB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
import surprise

In [0]:
def bin_searcher(elem, sorted_list, asc=True):
    
    if asc!=True:
        
        sorted_list= sorted_list.reverse()
    
    l=len(sorted_list)
    
    
    upper=l
    
    mp=int(l/2)
    
    lower=0
    
    count=0
    
    while upper>lower+1 and count<100:

        count+=1
        val= sorted_list[mp]

        if elem==val:
            return mp

        elif elem>val:
            lower=mp
            mp=int((lower+upper)/2)

        if elem<val:
            upper=mp
            mp= int((lower+upper)/2)
            
            
    return -1

In [0]:
bus=pd.read_csv('businesses.csv')

In [0]:
del bus['Unnamed: 0']

In [7]:
bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",NaN,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [0]:
bus=bus[bus.city=='Toronto']

In [0]:
bus['categories']=bus['categories'].fillna('lol')

In [0]:
bus=bus[bus.categories.str.contains('Restaurant')]

In [16]:
bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,1170 Queen Street W,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",Toronto,"{'Monday': '8:0-21:0', 'Tuesday': '8:0-21:0', ...",1,43.642889,-79.425429,Bolt Fresh Bar,M6J 1J5,57,3.0,ON
1,1051 Bloor Street W,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,"Restaurants, Nightlife, Breakfast & Brunch, Ve...",Toronto,"{'Tuesday': '9:0-18:0', 'Wednesday': '9:0-18:0...",0,43.660494,-79.432099,The Steady Cafe & Bar,M6H 1M4,29,3.5,ON
2,582 College Street,"{'Alcohol': ""u'full_bar'"", 'Caters': 'False', ...",mlHC2XcU9Bows6cnYEmRgg,"Restaurants, Breakfast & Brunch, Bars, Modern ...",Toronto,"{'Thursday': '18:0-2:0', 'Friday': '18:0-2:0',...",0,43.655420,-79.413352,Mad Crush Wine Bar,M6G 1B3,9,4.0,ON
3,619 Mount Pleasant Road,"{'RestaurantsPriceRange2': '2', 'HasTV': 'Fals...",RFbMVekR8lU9tPJ8sWrwHg,"Italian, Restaurants",Toronto,NaN,1,43.704229,-79.388230,Tavolino,M4S 2M5,18,4.0,ON
4,1614 Queen Street E,"{'RestaurantsGoodForGroups': 'True', 'Restaura...",NPHZkn1e-tSJAbo8Zm9rYw,"Tex-Mex, Mexican, Restaurants",Toronto,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,43.666181,-79.316468,Burrito Bandidos,M4L 1G4,43,3.5,ON


In [17]:
sum(bus.is_open)

5254

In [0]:
bus=bus[bus.is_open>0.5]

In [0]:
bus=bus.reset_index(drop=True)

In [0]:
del bus['attributes']
del bus['city']
del bus['state']

In [0]:
del bus['is_open']

In [41]:
bus.head()

,address,business_id,categories,hours,latitude,longitude,name,postal_code,review_count,stars
3408,1218 Saint Clair Avenue W,--DaPTJW3-tB1vP-PfdTEg,"Restaurants, Breakfast & Brunch","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",43.677807,-79.444674,Sunnyside Grill,M6E,49,3.5
3127,4917 Bathurst,-0aOudcaAyac0VJbMX-L1g,"Pizza, Middle Eastern, Restaurants",NaN,43.773238,-79.442641,Express Pizza & Grill,M2R 1X8,15,3.0
4023,613 King Street W,-3iE3lPp18Tn2ixqIRJB_w,"Salad, Juice Bars & Smoothies, Food, Coffee & ...","{'Monday': '7:30-20:30', 'Tuesday': '7:30-20:3...",43.644069,-79.400796,Iq Food,M5V 1M5,7,4.0
4391,1240 Bay Street,-4uT6QFVaRtc2Zmxvtd9Vg,"Barbeque, Fast Food, Restaurants","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",43.670324,-79.390681,BBQ Express,M5R 3A7,4,5.0
4420,1406 Queen St W,-6sE5v4OrVGmWxdIrbs-RQ,"Sushi Bars, Tapas/Small Plates, Restaurants, J...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-22:30',...",43.641153,-79.434475,Sho Izakaya,M6K 1L9,28,4.0


In [0]:
revs= pd.read_csv('yelper.csv')

In [0]:
del revs['Unnamed: 0']

In [26]:
revs.head()

,business_id,cool,date,elem,funny,review_id,stars,useful,user_id
0,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,43417,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,0,dacAIZ6fTM6mqwW5uxkskg
1,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,39447,0,fdiNeiN_hoCxCMy2wTRW9g,4.0,0,w31MKYsNFMrjhWxxAb5wIw
2,3fw2X5bZYeW9xCz_zGhOHg,5,2016-05-07 01:21:02,4337,4,G7XHMxG0bx9oBJNECG4IFg,3.0,5,jlu4CztcSxrKx56ba1a5AQ
3,zvO-PJCpNk4fgAVUnExYAA,1,2010-10-05 19:12:35,59316,1,8e9HxxLjjqc9ez5ezzN7iQ,1.0,3,d6xvYpyzcfbF_AZ8vMB7QA
4,8mIrX_LrOnAqWsB5JrOojQ,0,2011-11-30 02:11:15,9028,0,kbtscdyz6lvrtGjD1quQTg,4.0,0,FIk4lQQu1eTe2EpzQ4xhBA


In [0]:
bus=bus.sort_values(by='business_id', ascending=True)

In [0]:
b_list= list(bus.business_id)

In [0]:
rev_ids= list(revs.business_id)

In [0]:
id_nums=[]
for r in rev_ids:
  id_nums.append(bin_searcher(r, b_list))

In [34]:
len(id_nums)- len(revs)

0

In [0]:
revs['biz_num']=id_nums

In [0]:
revs=revs[revs.biz_num>=0]

In [37]:
len(revs)

305173

In [44]:
dict(bus.iloc[0])

{'address': '1218 Saint Clair Avenue W',
 'business_id': '--DaPTJW3-tB1vP-PfdTEg',
 'categories': 'Restaurants, Breakfast & Brunch',
 'hours': "{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', 'Wednesday': '7:0-15:0', 'Thursday': '7:0-15:0', 'Friday': '7:0-16:0', 'Saturday': '7:0-16:0', 'Sunday': '7:0-16:0'}",
 'latitude': 43.67780689999999,
 'longitude': -79.44467420000001,
 'name': 'Sunnyside Grill',
 'postal_code': 'M6E',
 'review_count': 49,
 'stars': 3.5}

In [0]:
bus_dicts={}
for i in range(len(bus)):
  b= dict(bus.iloc[i])
  bid= b['business_id']
  del b['business_id']
  bus_dicts[bid]=b

In [0]:
revs=revs.merge(bus,how='left', on='business_id', suffixes=('_revs', '_bus'))

In [51]:
revs.head()

,business_id,cool,date,elem,funny,review_id,stars_revs,useful,user_id,biz_num,address,categories,hours,latitude,longitude,name,postal_code,review_count,stars_bus
0,AakkkTuGZA2KBodKi2_u8A,0,2012-07-16 00:37:14,10766,1,JVcjMhlavKKn3UIt9p9OXA,1.0,1,TpyOT5E16YASd7EWjLQlrw,943,1603 Dundas Street W,"Asian Fusion, Restaurants, Vietnamese","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",43.649674,-79.435116,Pho Phuong,M6K 1T9,55,3.5
1,YvrylyuWgbP90RgMqZQVnQ,0,2017-04-07 21:27:49,33371,0,svK3nBU7Rk8VfGorlrN52A,5.0,0,NJlxGtouq06hhC7sS2ECYw,2934,232 Wellington Street W,"Sandwiches, Restaurants, Italian, Pizza","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",43.645277,-79.389704,Jz's Pizza,M5V 3W1,71,4.0
2,y-Iw6dZflNix4BdwIyTNGA,0,2014-06-27 21:19:23,57511,0,4bUyL7lzoWzDZaJETAKREg,3.0,0,_N7Ndn29bpll_961oPeEfw,5099,2876 Dundas Street W,"Restaurants, Food, Breweries, Gastropubs, Pizza","{'Monday': '17:0-1:0', 'Tuesday': '12:0-1:0', ...",43.665441,-79.465238,Indie Ale House,M6P 1Y9,235,4.0
3,YSUcHqlKMPHHJ_cTrqtNrA,0,2017-12-29 13:55:19,32926,0,E6B-2U2sGG3xgmnNWZAEew,1.0,0,DbccYu3OppWKl21OanZnTg,2894,32 Wellington Street E,"Italian, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",43.648404,-79.375469,Vagabondo Italian Ristorante+Lounge,M5E 1C6,77,3.0
4,TVLAvAFx0FqAkKGKHVjHpQ,1,2017-12-15 22:49:01,28247,0,6JgzKIDvophsU1yfK0DdyA,4.0,1,mZ1gXzL6Tn5Oky8_j0Kp7g,2479,140 Spadina Avenue,"Soup, Juice Bars & Smoothies, Restaurants, Foo...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",43.647726,-79.396247,Freshii,M5V 2L4,40,3.0


In [0]:
revs.to_csv('toronto_reviews.csv')

In [54]:
revs.describe()

,cool,elem,funny,stars_revs,useful,biz_num,latitude,longitude,review_count,stars_bus
count,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000,305173.000000
mean,0.472843,30245.505143,0.342786,3.649632,1.040610,2670.604090,43.673505,-79.390846,221.967284,3.649160
std,1.967094,16963.219643,1.446278,1.263013,2.590107,1501.965162,0.042646,0.046761,279.705114,0.550063
min,0.000000,37.000000,0.000000,1.000000,0.000000,1.000000,43.592327,-79.703390,3.000000,1.000000
25%,0.000000,15489.000000,0.000000,3.000000,0.000000,1382.000000,43.648752,-79.410044,61.000000,3.500000
50%,0.000000,31032.000000,0.000000,4.000000,0.000000,2719.000000,43.656345,-79.391912,134.000000,3.500000
75%,0.000000,44831.000000,0.000000,5.000000,1.000000,3961.000000,43.673749,-79.378802,273.000000,4.000000
max,177.000000,59372.000000,124.000000,5.000000,188.000000,5253.000000,43.876501,-79.139353,2121.000000,5.000000


In [58]:
bus.head()

,address,business_id,categories,hours,latitude,longitude,name,postal_code,review_count,stars
3408,1218 Saint Clair Avenue W,--DaPTJW3-tB1vP-PfdTEg,"Restaurants, Breakfast & Brunch","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",43.677807,-79.444674,Sunnyside Grill,M6E,49,3.5
3127,4917 Bathurst,-0aOudcaAyac0VJbMX-L1g,"Pizza, Middle Eastern, Restaurants",NaN,43.773238,-79.442641,Express Pizza & Grill,M2R 1X8,15,3.0
4023,613 King Street W,-3iE3lPp18Tn2ixqIRJB_w,"Salad, Juice Bars & Smoothies, Food, Coffee & ...","{'Monday': '7:30-20:30', 'Tuesday': '7:30-20:3...",43.644069,-79.400796,Iq Food,M5V 1M5,7,4.0
4391,1240 Bay Street,-4uT6QFVaRtc2Zmxvtd9Vg,"Barbeque, Fast Food, Restaurants","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",43.670324,-79.390681,BBQ Express,M5R 3A7,4,5.0
4420,1406 Queen St W,-6sE5v4OrVGmWxdIrbs-RQ,"Sushi Bars, Tapas/Small Plates, Restaurants, J...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-22:30',...",43.641153,-79.434475,Sho Izakaya,M6K 1L9,28,4.0


In [0]:
bus.to_csv('toronto_restaurants.csv')

In [0]:
b_means=revs.groupby('business_id').mean()

In [62]:
b_means.head()

,cool,elem,funny,stars_revs,useful,biz_num,latitude,longitude,review_count,stars_bus
business_id,,,,,,,,,,
-0aOudcaAyac0VJbMX-L1g,0.000000,37.0,0.066667,3.133333,1.333333,1.0,43.773238,-79.442641,15.0,3.0
-3iE3lPp18Tn2ixqIRJB_w,9.000000,78.0,3.714286,4.000000,9.857143,2.0,43.644069,-79.400796,7.0,4.0
-4uT6QFVaRtc2Zmxvtd9Vg,1.000000,92.0,0.000000,4.750000,1.250000,3.0,43.670324,-79.390681,4.0,5.0
-6sE5v4OrVGmWxdIrbs-RQ,0.785714,108.0,0.071429,4.250000,1.178571,4.0,43.641153,-79.434475,28.0,4.0
-7BCZH437U5FjmNJ26llkg,0.375000,113.0,0.277778,4.347222,0.861111,5.0,43.679767,-79.341605,69.0,4.5


In [0]:
b_means.reset_index(inplace=True)

In [0]:
b_means=b_means[['business_id', 'stars_revs']]

In [68]:
b_means

,business_id,stars_revs
0,-0aOudcaAyac0VJbMX-L1g,3.133333
1,-3iE3lPp18Tn2ixqIRJB_w,4.000000
2,-4uT6QFVaRtc2Zmxvtd9Vg,4.750000
3,-6sE5v4OrVGmWxdIrbs-RQ,4.250000
4,-7BCZH437U5FjmNJ26llkg,4.347222
5,-AVRReI-nfsa0lKlehEojw,3.235294
6,-BbnAc9YEO6pjvJGEtFbVQ,4.211155
7,-CLTZ0cO-ooKDDmXHFoGMg,3.821429
8,-ClafaRWkSRTfieIzO4r3Q,3.314815
9,-DOE4Qs8IMfgkvshYKfC7w,3.117647


In [0]:
bus= bus.merge(b_means, on='business_id')

In [0]:
bus['stars']=bus.stars_revs

In [0]:
del bus['stars_revs']

In [0]:
stars_only=bus[['business_id', 'stars']]

In [0]:
revs=revs.merge(stars_only,how='left', on='business_id', suffixes=('_revs', '_bus'))

In [77]:
revs.head()

,business_id,cool,date,elem,funny,review_id,stars_revs,useful,user_id,biz_num,address,categories,hours,latitude,longitude,name,postal_code,review_count,stars_bus,stars
0,AakkkTuGZA2KBodKi2_u8A,0,2012-07-16 00:37:14,10766,1,JVcjMhlavKKn3UIt9p9OXA,1.0,1,TpyOT5E16YASd7EWjLQlrw,943,1603 Dundas Street W,"Asian Fusion, Restaurants, Vietnamese","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",43.649674,-79.435116,Pho Phuong,M6K 1T9,55,3.5,3.303571
1,YvrylyuWgbP90RgMqZQVnQ,0,2017-04-07 21:27:49,33371,0,svK3nBU7Rk8VfGorlrN52A,5.0,0,NJlxGtouq06hhC7sS2ECYw,2934,232 Wellington Street W,"Sandwiches, Restaurants, Italian, Pizza","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",43.645277,-79.389704,Jz's Pizza,M5V 3W1,71,4.0,3.957746
2,y-Iw6dZflNix4BdwIyTNGA,0,2014-06-27 21:19:23,57511,0,4bUyL7lzoWzDZaJETAKREg,3.0,0,_N7Ndn29bpll_961oPeEfw,5099,2876 Dundas Street W,"Restaurants, Food, Breweries, Gastropubs, Pizza","{'Monday': '17:0-1:0', 'Tuesday': '12:0-1:0', ...",43.665441,-79.465238,Indie Ale House,M6P 1Y9,235,4.0,4.082988
3,YSUcHqlKMPHHJ_cTrqtNrA,0,2017-12-29 13:55:19,32926,0,E6B-2U2sGG3xgmnNWZAEew,1.0,0,DbccYu3OppWKl21OanZnTg,2894,32 Wellington Street E,"Italian, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",43.648404,-79.375469,Vagabondo Italian Ristorante+Lounge,M5E 1C6,77,3.0,3.089744
4,TVLAvAFx0FqAkKGKHVjHpQ,1,2017-12-15 22:49:01,28247,0,6JgzKIDvophsU1yfK0DdyA,4.0,1,mZ1gXzL6Tn5Oky8_j0Kp7g,2479,140 Spadina Avenue,"Soup, Juice Bars & Smoothies, Restaurants, Foo...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",43.647726,-79.396247,Freshii,M5V 2L4,40,3.0,2.975610


In [0]:
revs['stars_bus']=revs.stars

In [0]:
del revs['stars']

In [0]:
revs.to_csv('toronto_reviews.csv')

In [0]:
bus.to_csv('toronto_restaurants.csv')